# Install Packages and Setup Variables


In [1]:
!pip install -q llama-index==0.12.12 openai==1.70.0 llama-index-finetuning==0.3.1 llama-index-postprocessor-cohere-rerank==0.3.0 llama-index-embeddings-huggingface==0.5.1 \
                llama-index-embeddings-cohere==0.4.0 cohere==5.15.0 tiktoken==0.8.0 chromadb==1.0.15 llama-index-vector-stores-chroma==0.4.2 llama-index-llms-gemini==0.4.1 jedi==0.19.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.1/599.1 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 78.0 MB/s eta 0:0

In [2]:
import os

# Set the following API Keys in the Python environment. Will be used later.
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
os.environ["GOOGLE_API_KEY"] = "YOUR_GOOGLE_API_KEY"
os.environ["CO_API_KEY"] = "YOUR_COHERE_API_KEY"
cohere_key = os.environ["CO_API_KEY"]

# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')
# os.environ["GOOGLE_API_KEY"] = userdata.get('Google_api_key')
# os.environ["CO_API_KEY"] = userdata.get('CO_API_KEY')
# cohere_key = userdata.get('CO_API_KEY')

In [3]:
# Allows running asyncio in environments with an existing event loop, like Jupyter notebooks.

import nest_asyncio

nest_asyncio.apply()

# Load a Model


In [4]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(temperature=1, model="gpt-4o-mini")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

### Download vector store

In [5]:
from huggingface_hub import hf_hub_download

vectorstore = hf_hub_download(repo_id="jaiganesan/ai_tutor_knowledge", filename="vectorstore.zip", repo_type="dataset", local_dir=".")

vectorstore.zip:   0%|          | 0.00/97.2M [00:00<?, ?B/s]

In [6]:
!unzip -o vectorstore.zip

Archive:  vectorstore.zip
   creating: ai_tutor_knowledge/
   creating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/length.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/index_metadata.pickle  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/link_lists.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/header.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/data_level0.bin  
  inflating: ai_tutor_knowledge/chroma.sqlite3  


In [7]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import VectorStoreIndex

# Create the vector index
db = chromadb.PersistentClient(path="./ai_tutor_knowledge")
chroma_collection = db.get_or_create_collection("ai_tutor_knowledge")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(vector_store)

# Query Dataset


In [8]:

from llama_index.postprocessor.cohere_rerank import CohereRerank

# Define the Cohere Reranking object to return only the first two highest ranking chunks.

#cohere_rerank = CohereRerank(top_n=2, api_key=cohere_key)
cohere_rerank3 = CohereRerank(top_n=2, api_key=cohere_key,model = 'rerank-english-v3.0')

In [9]:
# Define the ServiceCotext object to tie the LLM for generating final answer,
# and the embedding model to help with retrieving related nodes.
# The `node_postprocessors` function will be applied to the retrieved nodes.
query_engine = index.as_query_engine(
    similarity_top_k=5, node_postprocessors=[cohere_rerank3]
)

res = query_engine.query("Write about Retrieval Augmented Generation?")

In [10]:
res.response

"Retrieval Augmented Generation (RAG) combines the strengths of information retrieval and generation processes to enhance the performance of large language models (LLMs). RAG systems primarily consist of two components: retrieval and generation.\n\nThe retrieval component extracts relevant information from external knowledge sources through two main phases: indexing and searching. Indexing organizes documents to facilitate efficient retrieval, utilizing methods like inverted indexes for sparse retrieval and dense vector encoding for dense retrieval. The searching phase employs these indexes to find pertinent documents based on user queries, often enhanced by rerankers to improve the relevance and ranking of the results.\n\nThe generation component utilizes the content retrieved alongside the user's query to formulate coherent and contextually relevant responses. This process can employ prompting techniques, such as Chain of Thought and Tree of Thought, which guide the generation toward

In [11]:
# Show the retrieved nodes
for src in res.source_nodes:
    print("Node ID\t", src.node_id)
    print("Title\t", src.metadata["title"])
    print("Text\t", src.text)
    print("Score\t", src.score)
    print("-_" * 20)

Node ID	 1c324686-fad7-4a41-bfd3-d44f9612ca91
Title	 Evaluation of Retrieval-Augmented Generation: A Survey:Research Paper Information
Text	 Authors: Hao Yu, Aoran Gan, Kai Zhang, Shiwei Tong, Qi Liu, and Zhaofeng Liu.Numerous studies of Retrieval-Augmented Generation (RAG) systems have emerged from various perspectives since the advent of Large Language Models (LLMs). The RAG system comprises two primary components: Retrieval and Generation. The retrieval component aims to extract relevant information from various external knowledge sources. It involves two main phases, indexing and searching. Indexing organizes documents to facilitate efficient retrieval, using either inverted indexes for sparse retrieval or dense vector encoding for dense retrieval. The searching component utilizes these indexes to fetch relevant documents based on the user's query, often incorporating the optional rerankers to refine the ranking of the retrieved documents. The generation component utilizes the retr

# Evaluate Cohere rerank

In [12]:
from huggingface_hub import hf_hub_download
from llama_index.core.evaluation import EmbeddingQAFinetuneDataset

# Download the evaluation dataset
hf_hub_download(repo_id="jaiganesan/ai_tutor_knowledge", filename="rag_eval_dataset_question_context_subset_50.json", repo_type="dataset", local_dir="./")
rag_eval_dataset = EmbeddingQAFinetuneDataset.from_json("./rag_eval_dataset_question_context_subset_50.json")

(…)_dataset_question_context_subset_50.json: 0.00B [00:00, ?B/s]

In [13]:
import pandas as pd


#  A simple function to show the evaluation result.
def display_results_retriever(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    hit_rate = full_df["hit_rate"].mean()
    mrr = full_df["mrr"].mean()

    metric_df = pd.DataFrame(
        {"Retriever Name": [name], "Hit Rate": [hit_rate], "MRR": [mrr]}
    )

    return metric_df

In [14]:
from llama_index.core.evaluation import RetrieverEvaluator

# We can evaluate the retievers with different top_k values.
for i in [2, 4, 6, 8, 10]:
    retriever = index.as_retriever(
        similarity_top_k=i, node_postprocessors=[cohere_rerank3]
    )
    retriever_evaluator = RetrieverEvaluator.from_metric_names(
        ["mrr", "hit_rate"], retriever=retriever
    )
    eval_results = await retriever_evaluator.aevaluate_dataset(rag_eval_dataset)
    print(display_results_retriever(f"Retriever top_{i}", eval_results))

    Retriever Name  Hit Rate   MRR
0  Retriever top_2       0.6  0.57
    Retriever Name  Hit Rate       MRR
0  Retriever top_4      0.68  0.593333
    Retriever Name  Hit Rate       MRR
0  Retriever top_6       0.7  0.597333
    Retriever Name  Hit Rate       MRR
0  Retriever top_8      0.78  0.608405
     Retriever Name  Hit Rate       MRR
0  Retriever top_10       0.8  0.610627
